In [5]:
import os
import pickle
import shutil
import Queue

import networkx as nx
import numpy as np
import pandas as pd

# from ghsom import main_no_labels as ghsom_main
from ghsom_parallel import main as ghsom_main

def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
# def add(d, key, value):
#     if key in d:
#         d[key].append(value)
#     else:
#         d.update({key : [value]})
    
root_dir = "/home/david/Documents/ghsom/hierarchical_exploration"

# for data in ["yeast_reactome", "yeast_uetz", "collins", "ccsb", "ito_core", "lc_multiple"]:
for data in ["yeast_reactome"]:
    
    print "data={}".format(data)
    print
    
#     for e_sg in [0.7, 0.6, 0.5, 0.4]:
    for e_sg in [0.7]:
        
        print "e_sg={}".format(e_sg)
        print

#         for e_en in [0.7, 0.6, 0.5, 0.4, 0.3]:
        for e_en in [0.6]:

            print "e_en={}".format(e_en)
            print

            os.chdir(root_dir)

            #ghsom parameters
            params = {'eta': 0.001,
                     'sigma': 1,
                      'e_sg': e_sg,
                     'e_en': e_en}

            map_file = '{}_hierarchy_communities_{}_{}'.format(data, e_sg, e_en)

            if not os.path.isfile("{}.pkl".format(map_file)):

                #run ghsom and save output
                print "running GHSOM and saving to {}.pkl".format(map_file)



#                 G, map = ghsom_main(params, '../embedded_{}.gpickle'.format(data), lam=1000)
#                 save_obj((G, map), map_file)
#                 print '\nnumber of communities found: {}, saved maps to {}'.format(len(map), map_file)

                G, networks = ghsom_main(params, '../embedded_{}.gpickle'.format(data), lam=1000, num_threads=5)
                save_obj((G, networks), map_file)
                print '\nnumber of maps grown: {}, saved maps to {}'.format(len(networks), map_file)



            else:

                print "{}.pkl already exists, loading maps".format(map_file)    
                #load output
#                 G, map = load_obj(map_file)
                G, networks = load_obj(map_file)

            #save results to file
            dir_name = "{}_hierarchy_communities_{}_{}".format(data, e_sg, e_en)
            if not os.path.isdir(dir_name):
#                 shutil.rmtree(dir_name)

                os.mkdir(dir_name)
                print 'made directory {}'.format(dir_name)

            os.chdir(dir_name)
            print "moved to {}".format(dir_name)

            #all genes
            all_genes_file = "all_genes.txt"
            with open(all_genes_file, 'w') as f:
                for n, d in G.nodes(data=True):
                    f.write("{}\n".format(n))
            print "wrote {}".format(all_genes_file)
            
            genes = G.nodes()
            gene_assignments = {k: v for k, v in zip(genes, 
                np.array([["" for j in range(10)] for i in range(len(genes))], dtype="S20"))}
            
            for id, network, e in networks:

                #shortest path matrix
                shortest_path = nx.floyd_warshall_numpy(network)
                communities_in_this_map = np.array([v for k, v in nx.get_node_attributes(network, "ID").items()])
                shortest_path_df = pd.DataFrame(shortest_path, index=communities_in_this_map)
                shortest_path_file = "{}_shortest_path.csv".format(id)
                
                shortest_path_df.to_csv(shortest_path_file, index=True, header=False, sep=',')
                print 'wrote shortest path matrix and saved as {}'.format(shortest_path_file)

                for n, d in network.nodes(data=True):
                    
                    community_assignment = d["ID"]
                    layer = community_assignment.count("-")
                    
                    for node in d["ls"]:
                        gene_assignments[node][layer] = community_assignment

#             #map queue
#             q = Queue.Queue()

#             c = 1
#             depth = 0
#             q.put((c, depth, map))

#             genes = G.nodes()
#             gene_assignments = {k: v for k, v in zip(genes, 
#                 np.array([["" for j in range(10)] for i in range(len(genes))], dtype="S20"))}

#             while not q.empty():

#                 map_id, depth, map = q.get()
#                 c = 1

#                 #shortest path matrix
#                 communities_in_this_map = np.array(["{}-{}".format(str(map_id).zfill(2), 
#                                                                    str(i).zfill(2)) for i in range(c, c + len(map))])

#                 shortest_path = nx.floyd_warshall_numpy(map).astype(np.int)
#     #             shortest_path = np.insert(shortest_path, 0, communities_in_this_map, axis=1)
#                 shortest_path_df = pd.DataFrame(shortest_path, index=communities_in_this_map)
#                 shortest_path_file = "{}_shortest_path.csv".format(str(map_id).zfill(2))
#     #             np.savetxt(shortest_path_file, shortest_path, fmt='%i', delimiter=",")
#                 shortest_path_df.to_csv(shortest_path_file, index=True, header=False, sep=',')
#                 print 'wrote shortest path matrix and saved as {}'.format(shortest_path_file)



#                 #gene community assignments
#                 for n, d in map.nodes(data=True):

#                     community = communities_in_this_map[c - 1]

#                     for node in d['ls']:
#                         gene_assignments[node][depth] = community

#                     #add map to queue
#                     m = d['n']

#                     if not m == []:

#                         q.put((community, depth + 1, m))   

#                     c += 1

            #back to matrix
            assignment_matrix = np.array([v for k, v in gene_assignments.items()])
            #remove unnecessary columns
            mask = assignment_matrix != ""
            idx = mask.any(axis = 0)
            assignment_matrix = assignment_matrix[:,idx]
            assignment_matrix = np.insert(assignment_matrix, 0, "01", axis=1)

            assignment_df = pd.DataFrame(assignment_matrix, index=genes)

            assignment_file = "assignment_matrix.csv"
        
            assignment_df.to_csv(assignment_file, index=True, header=False, sep=',')

#             np.savetxt(assignment_matrix_file, assignment_matrix, delimiter=",", fmt="%s")
            print "wrote assignment matrix and saved it as {}".format(assignment_file)
            print

data=yeast_reactome

e_sg=0.7

e_en=0.6

running GHSOM and saving to yeast_reactome_hierarchy_communities_0.7_0.6.pkl
MQE_0=3.97843041312, growth target=2.78490128918
MQE=3.04584977041, size of map=3
MQE=2.73352011157, size of map=4
growth terminated, MQE: 2.73352011157, target: 2.78490128918, number of deleted neurons: 0
submitted job: ID=01-02, e=2.66630454058, number of nodes=69
MQE_0=2.66630454058, growth target=1.8664131784submitted job: ID=01-04, e=3.84544976299, number of nodes=230

MQE_0=3.84544976299, growth target=2.6918148341
MQE=1.80764195423, size of map=3
growth terminated, MQE: 1.80764195423, target: 1.8664131784, number of deleted neurons: 0
submitted job: ID=01-02-01, e=2.19961639394, number of nodes=26
submitted job: ID=01-02-02, e=1.82769047025, number of nodes=23
MQE_0=2.19961639394, growth target=1.53973147576MQE_0=1.82769047025, growth target=1.27938332918

MQE=1.25043478186, size of map=3
growth terminated, MQE: 1.25043478186, target: 1.27938332918, number of del